# 1. Installation


In [ ]:
!pip install langchain langchain-community chromadb sentence-transformers langchain-text-splitters
!pip install -U langchain-ollama
!pip install -U langchain-huggingface
!pip install -U langchain-chroma
!pip install pypdf
!pip install langchain-mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.5 MB/s eta 0:00:

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
import gradio as gr
from uuid import uuid4

# 2. Data Path


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
DATA_PATH = "/content/drive/MyDrive/Forage/RAG_Files"    #You must change the data path according to how your file path on your Drive is structured, paste your correct data path


CHROMA_PATH = r"chroma_db"

# 3. loading the PDF document

In [13]:

loader = PyPDFDirectoryLoader(DATA_PATH)

In [14]:
raw_documents = loader.load()

In [15]:
raw_documents

[Document(metadata={'producer': 'Skia/PDF m142', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36', 'creationdate': '2025-11-26T20:58:02+00:00', 'title': 'Fiscal year analysis.ipynb - Colab', 'moddate': '2025-11-26T20:58:02+00:00', 'source': '/content/drive/MyDrive/Forage/RAG_Files/analysis_doc.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='1. Data Extraction from source\ue313\nimport pandas as pd\nfrom google.colab import drivedrive.mount(\'/content/drive\')\nDrive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/driv\nCompanyName FiscalYear TotalRevenue NetIncome TotalAssets TotalLiabilitiesCash Flow FromOperations\n0 Microsoft 2024 245,12288,136 512,163 243,686 118,548\n1 Microsoft 2023 211,195 72,361 411,976 205,753 87,582\n2 Microsoft 2022 198,27072,738 364,840 198,298 89,035\n3 Tesla 2024 97,690 7,153 122,070 48,390 14,923\n4 Tesla 2023 96,

# 4. Splitting the document

In [16]:
# splitting the document
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

split_docs = text_splitter.split_documents(raw_documents)

In [17]:
print("Chunks created:", len(split_docs))

Chunks created: 20


In [18]:
print("Example chunk:\n", split_docs[0].page_content[:500])

Example chunk:
 1. Data Extraction from source
import pandas as pd
from google.colab import drivedrive.mount('/content/drive')
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/driv
CompanyName FiscalYear TotalRevenue NetIncome TotalAssets TotalLiabilitiesCash Flow FromOperations
0 Microsoft 2024 245,12288,136 512,163 243,686 118,548
1 Microsoft 2023 211,195 72,361 411,976 205,753 87,582
2 Microsoft 2022 198,27072,738 364,840 198,298 89,035
3 Tesla 2024 97,690 


#5. Build Embedding Model


In [19]:
# Use HuggingFace embeddings (compatible with your existing setup)
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 6. creating unique ID's

In [20]:

uuids = [str(uuid4()) for _ in range(len(split_docs))]


In [21]:
print(uuids)

['d7ccba6b-0f20-4635-bab7-00ca62dcaf03', 'e8fc04f1-856a-4135-b7b3-73945ded69ec', 'b6231cbe-899f-4f55-9a0c-39d7d4febae7', 'e0f286e6-c5e2-44a5-b133-c0647b414fd3', 'fdc1c940-3578-471e-9e2c-c5b5b4e7ceba', 'd2e1b27e-573f-49a8-9de5-69fb4b1925cc', '52a0d0df-e18e-4a79-8aa8-26799f6ec724', '30057c0a-83dc-4271-a50e-e3b43de5b2e6', 'bbae3505-6556-457e-941c-0d58fc3cedf7', '7fdea68e-9256-4950-bef9-2f4010b44ff3', '1e4f64a9-d6aa-4419-ba14-cd1cea8c7bd0', '4d1d2719-ebfe-48b6-8451-98f8ab18536b', '5c774660-1ae5-49ee-b799-f7a0535c174e', 'cc024388-900e-4465-b2e4-4746290fb0b4', '113cc3fd-b63c-4bc9-862b-5d21bc28b46c', '5acfda36-d49f-42b1-a7c2-af33efd4cffb', 'fa58ef72-98e0-4b8b-b121-fc638b172c49', '4f34577e-5512-4dba-bca0-4850e9c45e49', '6d568379-50e8-4eb7-b40a-190b85d5efba', '97511412-1275-4aaf-bf50-94a4355a8082']


# 7. Initiating Vector Store

In [22]:
# Connect to the chromadb
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embed_model,
    persist_directory=CHROMA_PATH,
)

#8. Adding documents and unique ID's to vector store

In [23]:

stored_ids= vector_store.add_documents(documents=split_docs, ids=uuids)

#9. Set up the vectorstore to be the retriever

In [24]:

num_results = 5
retriever = vector_store.as_retriever(search_kwargs={'k': num_results})

#10. Building A RAG Chain

In [25]:
from google.colab import userdata

In [27]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-small-latest",
    api_key=userdata.get('Financial_Analysis')          # create your own mistral api key on mistral account and use colab secrets to store your api key.Provide you own secretName
)

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [29]:
from langchain_core.runnables import RunnableLambda # Added import

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful analyst. Use only the provided context."),
    ("human", "Question: {question}\n\nContext:\n{context}")
])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": (lambda x: x["question"]) | retriever | RunnableLambda(format_docs),
        "question": lambda x: x["question"]
    }
    | prompt
    | llm
    | StrOutputParser()
)

# 11. Testing RAG to check if it works

In [30]:
#response = rag_chain.invoke({"question": "What is the average annual growth rate of Tesla."})
#print(response)


Based on the provided context, the average annual growth rate of Tesla's revenue is 9.87%. This information is found in the table under "Average Annual Growth Rates by Company (%)" where it lists the revenue growth percentage for Tesla.


# 12. Initiate the Gradio app

In [31]:
# Function to query your RAG chain
def ask_rag_gradio(question):
    try:
        response = rag_chain.invoke({"question": question})
        return response
    except Exception as e:
        return f"Error: {e}"


In [32]:
iface = gr.Interface(
    fn=ask_rag_gradio,           # the function to call
    inputs=gr.Textbox(lines=2, placeholder="Type your question here..."),
    outputs=gr.Textbox(lines=10),
    title="Financial Analysis RAG Chatbot",
    description="Ask questions about the Financial of 3 Companies from their 3 Fiscal Years. The model will answer using the provided context."
)


# 13 Launching the chatbot
  if the shown url doesnt work it means it has expired, re-run the cell to get a new url for the chatbot

In [33]:
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bf9d362ee62dde4893.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
